In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#!ls "/content/gdrive/MyDrive/"

In [3]:
!pip install foolbox

In [4]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

In [5]:
from __future__ import print_function
import torch
import numpy as np
import warnings
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
import torchvision.models as models
import urllib


In [6]:
class Model_Drop(nn.Module):

    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

        self.drop_layer = nn.Dropout(p=0.5)
        self.drop_layer_conv = nn.Dropout2d(p=0.5)

    def last_hidden_layer_output(self, x):

        x = F.relu(self.conv1(x),inplace=True)
        x = F.max_pool2d(F.relu(self.conv2(x),inplace=True), kernel_size=2, stride=2)

        x = F.relu(self.conv3(x),inplace=True)
        x = F.max_pool2d(F.relu(self.conv4(x),inplace=True), kernel_size=2, stride=2)

        x = F.relu(self.conv5(x),inplace=True)
        x = self.drop_layer_conv(F.max_pool2d(F.relu(self.conv6(x),inplace=True), kernel_size=2, stride=2))

        x = x.view(-1, 4096)
        x = self.drop_layer(F.relu(self.fc1(x)))
        x = self.drop_layer(F.relu(self.fc2(x)))
        return x

    def forward(self, x):

        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x



In [7]:
print("Please chose reversal method you would like to test 0, 1, 2 :\n Epistemic Uncertainty Based (0)\n  Scibilic Uncertainty Based (1)\n Scibilic Uncertainty Based (For Pred Class only) (2)\n")

input_a = int(input())

Please chose reversal method you would like to test 0, 1, 2 :
 Epistemic Uncertainty Based (0)
  Scibilic Uncertainty Based (1)
 Scibilic Uncertainty Based (For Pred Class only) (2)

2


In [8]:
def norms(Z):
    """Compute norms over all but the first dimension"""
    return Z.view(Z.shape[0], -1).norm(dim=1)[:,None,None,None]

In [9]:
def unc_defense(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model((image+delta).clamp(0,1))
            output = softmax(output)

            dropout_predictions[i] = output


        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)


        model.eval()
        o = model((image + delta).clamp(0, 1))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta).clamp(0, 1))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)

    model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [10]:
def unc_defense_epistemic(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)
    torch.cuda.manual_seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model((image+delta).clamp(0,1))
            output = softmax(output)

            dropout_predictions[i] = output

        mean = torch.mean(dropout_predictions, dim=0)
        pred_mean = mean

        aleatoric = torch.zeros([item_count,10,10])
        epistemic = torch.zeros([item_count,10,10])

        for ti in range(50):

            pred_t = dropout_predictions[ti]

            aleatoric += torch.diag_embed(pred_t, offset=0, dim1=-2, dim2=-1) - pred_t[:, :, None] @ pred_t[:, None, :]
            epistemic += (pred_t - pred_mean)[:, :, None] @ (pred_t - pred_mean)[:, None, :]


        aleatoric = aleatoric / 50
        epistemic = epistemic / 50

        ep = epistemic.diagonal(dim1=-2, dim2=-1)[:].mean(1)
        al = aleatoric.diagonal(dim1=-2, dim2=-1)[:].mean(1)

        sc = ep / al
  

        var = ep
        var = var.reshape(1,item_count)
        var = var.to(device)

        model.eval()
        o = model((image + delta).clamp(0, 1))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta).clamp(0, 1))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [11]:
def unc_defense_scibilic(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)
    torch.cuda.manual_seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model((image+delta).clamp(0,1))
            output = softmax(output)

            dropout_predictions[i] = output

        mean = torch.mean(dropout_predictions, dim=0)
        pred_mean = mean

        aleatoric = torch.zeros([item_count,10,10])
        epistemic = torch.zeros([item_count,10,10])

        for ti in range(50):

            pred_t = dropout_predictions[ti]

            aleatoric += torch.diag_embed(pred_t, offset=0, dim1=-2, dim2=-1) - pred_t[:, :, None] @ pred_t[:, None, :]
            epistemic += (pred_t - pred_mean)[:, :, None] @ (pred_t - pred_mean)[:, None, :]


        aleatoric = aleatoric / 50
        epistemic = epistemic / 50

        ep = epistemic.diagonal(dim1=-2, dim2=-1)[:].mean(1)
        al = aleatoric.diagonal(dim1=-2, dim2=-1)[:].mean(1)

        sc = ep / al
        #sc = al / ep

        var = sc
        var = var.reshape(1,item_count)
        var = var.to(device)


        model.eval()
        o = model((image + delta).clamp(0, 1))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta).clamp(0, 1))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [12]:
def unc_defense_scibilic_pred(image,model,epsilon, num_iter, alpha,preds):

    torch.manual_seed(2)
    np.random.seed(2)
    torch.cuda.manual_seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])
        
        #torch.manual_seed(2)
        #np.random.seed(2)
        #torch.cuda.manual_seed(2)

        for i in range(50):

            enable_dropout(model)
            output = model((image+delta).clamp(0,1))
            output = softmax(output)

            dropout_predictions[i] = output

        mean = torch.mean(dropout_predictions, dim=0)
        pred_mean = mean

        aleatoric = torch.zeros([item_count,10,10])
        epistemic = torch.zeros([item_count,10,10])

        for ti in range(50):

            pred_t = dropout_predictions[ti]


            #print("item_count",item_count)
            #print("pred_t shape", pred_t.shape)
            #print("aleatoric shape", aleatoric.shape)
            #print("squezed shape", squezed_pred_t.shape)
            #print("outer product shape", (pred_t[:, :, None] @ pred_t[:, None, :]).shape)
            #print("diag size is ", torch.diag_embed(pred_t, offset=0, dim1=-2, dim2=-1).shape)

            aleatoric += torch.diag_embed(pred_t, offset=0, dim1=-2, dim2=-1) - pred_t[:, :, None] @ pred_t[:, None, :]
            epistemic += (pred_t - pred_mean)[:, :, None] @ (pred_t - pred_mean)[:, None, :]


        aleatoric = aleatoric / 50
        epistemic = epistemic / 50

        preds = preds.to(device)
        epistemic = epistemic.to(device)
        aleatoric = aleatoric.to(device)

        ep = epistemic.diagonal(dim1=-2, dim2=-1).gather(1, preds.unsqueeze(1))
        al = aleatoric.diagonal(dim1=-2, dim2=-1).gather(1, preds.unsqueeze(1))

        #ep = epistemic.diagonal(dim1=-2, dim2=-1)[:].mean(1)
        #al = aleatoric.diagonal(dim1=-2, dim2=-1)[:].mean(1)

        sc = ep / al

        var = sc
        var = var.reshape(1,item_count)
        var = var.to(device)

        model.eval()
        o = model((image + delta).clamp(0, 1))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            #print(torch.count_nonzero(delta_unc)/delta_unc.shape[0])
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)


        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta).clamp(0, 1))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            #print(torch.count_nonzero(delta_unc)/delta_unc.shape[0])
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

            #delta.data -= alpha * delta_unc / (norms(delta_unc) + 1e-30)
            #delta.data = torch.min(torch.max(delta.detach(), -image), 1 - image)  # clip X+delta to [0,1]
            #delta.data *= epsilon / norms(delta.detach()).clamp(min=epsilon)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [13]:
def norms(Z):
    """Compute norms over all but the first dimension"""
    return Z.view(Z.shape[0], -1).norm(dim=1)[:,None,None,None]


In [14]:
#train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=transforms.ToTensor())

#train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified


In [15]:
warnings.filterwarnings("ignore")

torch.manual_seed(2)
np.random.seed(2)

batch_size = 64
eps = 4.0/255
alpha = 0.2 * eps
num_iter = 10
eps_l2 = 0.42
#eps_l2 = 2
print(eps)

#eps_little = 0.2*eps
eps_little = 0.2/255
alpha_reverse = 0.2 * eps_little
num_iter_reverse = 10

count_successful_reverse = 0
count_unsuccessful_reverse = 0

count_successfull_attack = 0
count_unsuccessfull_attack = 0


0.01568627450980392


In [16]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#model_cnn_teacher = Model_Drop()
#model_cnn_teacher.load_state_dict(torch.load("/content/gdrive/MyDrive/TENET_CIFAR10_model_cnn_teacher_T_100.pt",map_location=device))
#model_cnn_teacher.eval()
#model_cnn_teacher.to(device)

model_cnn_student = Model_Drop()
model_cnn_student.load_state_dict(torch.load("/content/gdrive/MyDrive/TENET_CIFAR10_model_cnn_student_T_100.pt",map_location=device))
model_cnn_student.eval()
model_cnn_student.to(device)


#model_cnn_normal = Model_Drop()
#model_cnn_normal.load_state_dict(torch.load("/content/gdrive/MyDrive/TENET_CIFAR10_model_cnn_normal.pt",map_location=device))
#model_cnn_normal.eval()
#model_cnn_normal.to(device)


softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)


2021-07-01 14:34:30


In [17]:
 torch.__version__

'1.9.0+cu102'

In [18]:
for i, (image, label) in enumerate(test_loader):

    if i% 25 == 0:
      print("i is ",i)


    image = image.to(device)
    label = label.to(device)

    model_cnn_student.eval()
    with torch.no_grad():
        o = model_cnn_student(image)
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]


    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    #attack = L2CarliniWagnerAttack(steps=1000, confidence = 40)
    #fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    #pert = torch.tensor(clipped_advs[0])

    #attack = LinfBasicIterativeAttack()
    #fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    #attack = LinfPGD()
    #fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    #attack = LinfFastGradientAttack()
    #fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    attack = L2DeepFoolAttack()
    fmodel = PyTorchModel(model_cnn_student, bounds=(0, 1))
    raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    pert = torch.tensor(clipped_advs[0])
    
    # pert = image

    model_cnn_student.eval()
    with torch.no_grad():
        o = model_cnn_student(pert)
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]

    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]


    if inds_wrong_after_attack.shape[0] == 0:
        print("no successful attack at ith batch ,where i = ", i)
        continue

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]
    pred_pert = pred_pert[inds_wrong_after_attack]

    #reversed_pert = unc_defense(pert, model_cnn_student, eps_little, num_iter_reverse, alpha_reverse)

    if input_a == 0:
      reversed_pert = unc_defense_epistemic(pert, model_cnn_student, eps_little, num_iter_reverse, alpha_reverse)
    elif input_a == 1:
      reversed_pert = unc_defense_scibilic(pert, model_cnn_student, eps_little, num_iter_reverse, alpha_reverse)
    elif input_a == 2:
      reversed_pert = unc_defense_scibilic_pred(pert, model_cnn_student, eps_little, num_iter_reverse, alpha_reverse,pred_pert)

    model_cnn_student.eval()
    with torch.no_grad():
        o = model_cnn_student(reversed_pert)
        o = softmax(o)
    pred_reverse = o.data.max(1, keepdim=True)[1]

    pred_reverse = pred_reverse.view_as(label)

    inds_correct_after_reverse = np.where(pred_reverse.cpu() == label.cpu())[0]
    inds_wrong_after_reverse = np.where(pred_reverse.cpu() != label.cpu())[0]

    reversed_pert = reversed_pert[inds_correct_after_reverse]
    image = image[inds_correct_after_reverse]
    label = label[inds_correct_after_reverse]
    pert = pert[inds_correct_after_reverse]
    pred_pert = pred_pert[inds_correct_after_reverse]

    inds_correct_after_reverse = inds_correct_after_reverse.tolist()
    inds_wrong_after_reverse = inds_wrong_after_reverse.tolist()

    count_successful_reverse += len(inds_correct_after_reverse)
    count_unsuccessful_reverse += len(inds_wrong_after_reverse)

    #if i >= 3:
      #break
    
    #if i%10 == 0:
      #print(i)
      #break


i is  0
i is  25
i is  50
i is  75
i is  100
i is  125
i is  150


In [19]:
print("Number of successful reverse operation is : ", count_successful_reverse)
print("Number of unsuccessful reverse operation is : ", count_unsuccessful_reverse)

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful reverse operation is :  7472
Number of unsuccessful reverse operation is :  23
2021-07-01 14:53:25
